# Stacking Decorators: Applying Multiple Layers

- Python lets you attach more than one decorator to a single function by writing multiple `@decorator` lines above the `def`.  
- Each decorator contributes a distinct slice of behaviour (logging, timing, caching, auth checks) keeping the core function clean.

## Application vs. Execution Order

- **Decoration happens bottom‑up** when the function is defined:  
  1. Decorator nearest the `def` wraps the original first.  
  2. Each line above wraps the result of the previous decoration.  
- **Execution happens top‑down** (outside‑in) when the decorated function is called: the outermost wrapper runs first, then calls the inner wrapper, and so on until the original function runs.

## Order Matters

- Swapping decorator order changes both side‑effects and final result if wrappers transform the return value.

--- Call foo() (A outside, B inside) ---
A before
B before
  >>> inside foo
B after
A after

--- Call bar() (B outside, A inside) ---
B before
A before
  >>> inside bar
A after
B after


In [5]:
from functools import wraps

def decorator_A(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("A before")
        result = func(*args, **kwargs)
        print("A after")
        return result
    return wrapper


def decorator_B(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("B before")
        result = func(*args, **kwargs)
        print("B after")
        return result
    return wrapper


# The execution starts from the outermost function i.e. decorator_A
@decorator_A # then, this wraps the returned function by decorator_B and returns the final function
@decorator_B # first, this wraps the target function and returns the new function
def foo():
    print("  >>> inside function foo")


@decorator_B
@decorator_A
def bar():
    print("  >>> inside function bar")


foo()

print("----")

bar()

A before
B before
  >>> inside function foo
B after
A after
----
B before
A before
  >>> inside function bar
A after
B after


## Hands-on Exercise

In [19]:
from functools import wraps
 
def log_enter(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("ENTERING")
        result = func(*args, **kwargs)
        print(f"Result in log_enter: {result}")
        return result
 
    return wrapper
 
def log_exit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"Result in log_exit: {result}")
        print("EXITING")
        return result
 
    return wrapper
 
@log_enter
@log_exit
def perform_task():
    print("......TASK RUNNING......")
 
perform_task()

ENTERING
......TASK RUNNING......
Result in log_exit: None
EXITING
Result in log_enter: None
